In [6]:
import os

import torch
from fastbook import *

In [7]:

path = Path('/mnist')
path.ls()

(#2) [Path('/mnist/training'),Path('/mnist/testing')]

In [8]:
(path/"training").ls()

(#10) [Path('/mnist/training/4'),Path('/mnist/training/3'),Path('/mnist/training/9'),Path('/mnist/training/8'),Path('/mnist/training/0'),Path('/mnist/training/1'),Path('/mnist/training/5'),Path('/mnist/training/7'),Path('/mnist/training/6'),Path('/mnist/training/2')]

In [9]:
(path/"testing").ls()

(#10) [Path('/mnist/testing/4'),Path('/mnist/testing/3'),Path('/mnist/testing/9'),Path('/mnist/testing/8'),Path('/mnist/testing/0'),Path('/mnist/testing/1'),Path('/mnist/testing/5'),Path('/mnist/testing/7'),Path('/mnist/testing/6'),Path('/mnist/testing/2')]

In [10]:
training_ones = (path / "training" / "1").ls().sorted()
training_fives = (path / "training" / "5").ls().sorted()
testing_ones = (path / "testing" / "1").ls().sorted()
testing_fives = (path / "testing" / "5").ls().sorted()
training_ones, training_fives, testing_ones, testing_fives

((#6742) [Path('/mnist/training/1/10006.png'),Path('/mnist/training/1/10007.png'),Path('/mnist/training/1/1002.png'),Path('/mnist/training/1/10020.png'),Path('/mnist/training/1/10027.png'),Path('/mnist/training/1/1003.png'),Path('/mnist/training/1/10040.png'),Path('/mnist/training/1/10048.png'),Path('/mnist/training/1/10058.png'),Path('/mnist/training/1/10067.png')...],
 (#5421) [Path('/mnist/training/5/0.png'),Path('/mnist/training/5/100.png'),Path('/mnist/training/5/10008.png'),Path('/mnist/training/5/10015.png'),Path('/mnist/training/5/10030.png'),Path('/mnist/training/5/10035.png'),Path('/mnist/training/5/10049.png'),Path('/mnist/training/5/10051.png'),Path('/mnist/training/5/10056.png'),Path('/mnist/training/5/10062.png')...],
 (#1135) [Path('/mnist/testing/1/1004.png'),Path('/mnist/testing/1/1008.png'),Path('/mnist/testing/1/1011.png'),Path('/mnist/testing/1/1019.png'),Path('/mnist/testing/1/1025.png'),Path('/mnist/testing/1/1027.png'),Path('/mnist/testing/1/1030.png'),Path('/mni

In [11]:
import PIL.Image

img_1 = PIL.Image.open(training_ones[0])
img_1

In [12]:
tensor(img_1)

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  60, 255,  78,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  59, 253,  78,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 183, 253,  78,   0,  

In [13]:
training_ones_tensor = torch.stack([tensor(Image.open(img)) for img in training_ones]).float()/255
testing_ones_tensor = torch.stack([tensor(Image.open(img)) for img in testing_ones]).float()/255
training_ones_tensor.shape, testing_ones_tensor.shape

(torch.Size([6742, 28, 28]), torch.Size([1135, 28, 28]))

In [14]:
training_fives_tensor = torch.stack([tensor(Image.open(img)) for img in training_fives]).float()/255
testing_fives_tensor = torch.stack([tensor(Image.open(img)) for img in testing_fives]).float()/255
training_fives_tensor.shape, testing_fives_tensor.shape

(torch.Size([5421, 28, 28]), torch.Size([892, 28, 28]))

In [15]:
train_x = torch.cat([
    torch.reshape(training_ones_tensor, (len(training_ones_tensor), 28*28)),
    torch.reshape(training_fives_tensor, (len(training_fives_tensor), 28*28)),
    ])
train_x.shape

torch.Size([12163, 784])

In [16]:
train_y = torch.cat([torch.full((len(training_ones_tensor),), 1), torch.full((len(training_fives_tensor),), 0)])
train_y.shape

torch.Size([12163])

In [17]:
valid_x = torch.cat([
    torch.reshape(testing_ones_tensor, (len(testing_ones_tensor), 28*28)),
    torch.reshape(testing_fives_tensor, (len(testing_fives_tensor), 28*28)), 
])
valid_x.shape

torch.Size([2027, 784])

In [18]:
valid_y = torch.cat([torch.full((len(testing_ones_tensor),), 1), torch.full((len(testing_fives_tensor),), 0)])
valid_y.shape

torch.Size([2027])

In [19]:
def init_params(size, std=1.0):
    return (torch.randn(size) * std).requires_grad_()

In [20]:
def loss(pred: Tensor, target: Tensor):
    pred = pred.sigmoid()
    return torch.where(target==1, 1 - pred, pred).mean()
    

In [21]:
dl = DataLoader(list(zip(train_x, train_y)), batch_size=256)
xb, yb = first(dl)
xb.shape, yb.shape 

(torch.Size([256, 784]), torch.Size([256]))

In [22]:
valid_dl = DataLoader(list(zip(valid_x, valid_y)))

In [25]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    l = loss(preds, yb)
    l.backward()

class BasicOptim:
    
    def __init__(self, params: Tensor, lr: float):
        self.params = list(params)
        self.lr = lr
        
    def step(self):
        for p in self.params:
            # print(f'gradient: {p.grad.data[4:4:4]}')
            p.data -= p.grad.data * self.lr
            
    def zero_grad(self):
        for p in self.params:
            p.grad = None

def train_epoch(model, opt: BasicOptim):
    for xb, yb in dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()
        
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)


lr = 1.

linear_model = nn.Linear(28 * 28, 1)

opt = BasicOptim(linear_model.parameters(), lr)

for i in range(20):
    train_epoch(linear_model, opt)
    print(validate_epoch(linear_model))

0.6428
0.9156
0.9635
0.9798
0.9847
0.9872
0.9896
0.9901
0.9916
0.9931
0.9931
0.9931
0.9941
0.9941
0.9941
0.9941
0.9941
0.9941
0.9946
0.9946


In [26]:
linear_model(valid_x)

tensor([[  4.5224],
        [  6.7575],
        [  8.7884],
        ...,
        [ -5.4234],
        [-16.6684],
        [-12.8378]], grad_fn=<AddmmBackward0>)